# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [ ]:
import random

import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [ ]:
data = pd.read_csv('/Users/macbookair7/Documents/Irving/Profesional/DATA/Entregables/lab-intro-recommender-systems/data/customer_product_sales.csv')

In [10]:
data.shape

(68584, 7)

In [11]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [12]:
df = data.groupby(by=['CustomerID','ProductName'], as_index = False).agg({"Quantity":"sum"})
df.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [13]:
df_2 = data.pivot_table(index='ProductName',columns='CustomerID',
                       values='Quantity').fillna(0)
df_2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [14]:
from scipy.spatial.distance import pdist, squareform

In [15]:
distances = pd.DataFrame(1/(1 + squareform(pdist(df_2.T, 'euclidean'))), 
                         index=df_2.columns, columns=df_2.columns)

distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.005023,0.005032,0.005055,0.005036,0.005042,0.005042,0.005042,0.005039,0.005039,0.005033,...,0.003732,0.003945,0.003749,0.004045,0.003964,1.000000,0.003945,0.003945,0.003579,0.003608
98069,0.004880,0.004909,0.004894,0.004871,0.004876,0.004877,0.004882,0.004880,0.004886,0.004886,...,0.003638,0.003765,0.003716,0.003782,0.003888,0.003945,1.000000,0.003907,0.003608,0.003765
98159,0.005026,0.005042,0.005042,0.005042,0.005039,0.005039,0.005039,0.005033,0.005045,0.005036,...,0.003732,0.003984,0.003852,0.003888,0.003817,0.003945,0.003907,1.000000,0.003732,0.003732


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [16]:
def top_five_users():
  top_five_users.col = (list(data.CustomerID)[random.randint(1,data.shape[0])])
  print('The 5 users most similar for',top_five_users.col,'are:')
  top_five=distances[top_five_users.col]
  top_five=top_five.drop([top_five_users.col],axis=0)
  users = top_five.nlargest(5)
  return users

In [17]:
ex=top_five_users()
ex

The 5 users most similar for 19526 are:


CustomerID
264     0.026816
649     0.026816
3267    0.026807
1577    0.026767
1428    0.026679
Name: 19526, dtype: float64

In [18]:
user_ID = top_five_users.col
user_ID

19526

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [19]:
similar=df.loc[df['CustomerID'].isin(list(ex.index))]
print(similar.CustomerID.unique())
similar

[ 264  649 1428 1577 3267]


,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1
...,...,...,...
2729,3267,Wanton Wrap,1
2730,3267,"Water - Mineral, Natural",1
2731,3267,"Wine - Cahors Ac 2000, Clos",1
2732,3267,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [20]:
prd_top=similar.groupby('ProductName',as_index=False).agg({'Quantity':'sum'}).sort_values(by='Quantity',ascending=False,).reset_index(drop=True)
prd_top

,ProductName,Quantity
0,"Mushrooms - Black, Dried",4
1,Turkey - Oven Roast Breast,4
2,Cheese - Mozzarella,4
3,Beans - Kidney White,3
4,Rice - Long Grain,3
...,...,...
239,Loquat,1
240,Mayonnaise - Individual Pkg,1
241,Milk - 2%,1
242,Milk Powder,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [21]:
merged = prd_top.merge(df_2, on='ProductName')
merged_list=merged[merged[user_ID]==0].sort_values('Quantity', ascending = False)
merged_list

,ProductName,Quantity,33,200,264,356,412,464,477,639,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,"Mushrooms - Black, Dried",4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
2,Cheese - Mozzarella,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0
4,Rice - Long Grain,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,25.0,25.0
5,Quiche Assorted,3,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0
6,"Pork - Loin, Center Cut",3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Table Cloth 62x114 White,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138,"Tart Shells - Sweet, 4",1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139,Tea - Decaf Lipton,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140,Tea - English Breakfast,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,25.0


In [22]:
print(merged_list.ProductName[:5])

0    Mushrooms - Black, Dried
2         Cheese - Mozzarella
4           Rice - Long Grain
5             Quiche Assorted
6     Pork - Loin, Center Cut
Name: ProductName, dtype: object


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [23]:
recom = {}
ids = list(data.CustomerID.unique())[:10]

In [24]:
for i in ids:
  top_five=distances[i]
  top_five=top_five.drop([i],axis=0)
  users = top_five.nlargest(5)
  #print(users)
  similar=df.loc[df['CustomerID'].isin(list(users.index))]
  prd_top=similar.groupby('ProductName',as_index=False).agg({'Quantity':'sum'}).sort_values(by='Quantity',ascending=False,).reset_index(drop=True)
  merged = prd_top.merge(df_2, on='ProductName')
  merged_list=merged[merged[user_ID]==0].sort_values('Quantity', ascending = False)
  recommendations = list(merged_list.ProductName[:5].values)
  recom[i] = recommendations

In [25]:
recom

{61288: ['Mushrooms - Black, Dried',
  'Sponge Cake Mix - Chocolate',
  'Wine - Magnotta, Merlot Sr Vqa',
  'Beef - Rib Eye Aaa',
  'Chicken - Soup Base'],
 77352: ['Guinea Fowl',
  'Grenadine',
  'Ecolab - Mikroklene 4/4 L',
  'Coconut - Shredded, Sweet',
  'Beets - Candy Cane, Organic'],
 40094: ['Pasta - Orecchiette',
  'Quiche Assorted',
  'Oregano - Dry, Rubbed',
  'Tea - Decaf Lipton',
  'Liners - Banana, Paper'],
 23548: ['Spinach - Baby',
  'Banana Turning',
  'Tea - English Breakfast',
  'Flavouring - Orange',
  'Sauce - Demi Glace'],
 78981: ['Lettuce - Frisee',
  'Longos - Chicken Wings',
  'Pop Shoppe Cream Soda',
  'Crush - Cream Soda',
  'Sprouts - Alfalfa'],
 83106: ['Skirt - 29 Foot',
  'Cheese - Boursin, Garlic / Herbs',
  'Garlic - Peeled',
  'Soup - Campbells Tomato Ravioli',
  'Beef - Rib Eye Aaa'],
 11253: ['Smirnoff Green Apple Twist',
  'Watercress',
  'Tomatoes Tear Drop',
  'Lettuce - Treviso',
  'Juice - Cranberry, 341 Ml'],
 35107: ['Vinegar - Sherry',
  'Fla

##Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.¶


In [26]:
columns_names = ['Recommendation-1','Recommendation-2','Recommendation-3','Recommendation-4','Recommendation-5']


In [27]:
five_recom = pd.DataFrame.from_dict(recom, orient='index', dtype=None, columns= columns_names)
five_recom

,Recommendation-1,Recommendation-2,Recommendation-3,Recommendation-4,Recommendation-5
61288,"Mushrooms - Black, Dried",Sponge Cake Mix - Chocolate,"Wine - Magnotta, Merlot Sr Vqa",Beef - Rib Eye Aaa,Chicken - Soup Base
77352,Guinea Fowl,Grenadine,Ecolab - Mikroklene 4/4 L,"Coconut - Shredded, Sweet","Beets - Candy Cane, Organic"
40094,Pasta - Orecchiette,Quiche Assorted,"Oregano - Dry, Rubbed",Tea - Decaf Lipton,"Liners - Banana, Paper"
23548,Spinach - Baby,Banana Turning,Tea - English Breakfast,Flavouring - Orange,Sauce - Demi Glace
78981,Lettuce - Frisee,Longos - Chicken Wings,Pop Shoppe Cream Soda,Crush - Cream Soda,Sprouts - Alfalfa
83106,Skirt - 29 Foot,"Cheese - Boursin, Garlic / Herbs",Garlic - Peeled,Soup - Campbells Tomato Ravioli,Beef - Rib Eye Aaa
11253,Smirnoff Green Apple Twist,Watercress,Tomatoes Tear Drop,Lettuce - Treviso,"Juice - Cranberry, 341 Ml"
35107,Vinegar - Sherry,Flavouring - Orange,Wonton Wrappers,Ice Cream Bar - Hageen Daz To,"Spoon - Soup, Plastic"
15088,Smirnoff Green Apple Twist,Blueberries,"Pepper - Black, Whole",Wonton Wrappers,Wine - Wyndham Estate Bin 777
26031,Muffin Batt - Choc Chk,Beef - Top Sirloin,Knife Plastic - White,Cheese - Mozzarella,Olive - Spread Tapenade


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [28]:
distances_jaccard = pd.DataFrame(1/(1 + squareform(pdist(df_2.T, 'jaccard'))), 
                         index=df_2.columns, columns=df_2.columns)

recom_j={}

for i in ids:
  top_five_j=distances_jaccard[i]
  top_five_j=top_five_j.drop([i],axis=0)
  users_j = top_five_j.nlargest(5)
  similar_j=df.loc[df['CustomerID'].isin(list(users_j.index))]
  prd_top_j=similar_j.groupby('ProductName',as_index=False).agg({'Quantity':'sum'}).sort_values(by='Quantity',ascending=False,).reset_index(drop=True)
  merged_j = prd_top_j.merge(df_2, on='ProductName')
  merged_list_j=merged_j[merged_j[user_ID]==0].sort_values('Quantity', ascending = False)
  recommendations_j = list(merged_list_j.ProductName[:5].values)
  recom_j[i] = recommendations_j

In [29]:
five_recom_j = pd.DataFrame.from_dict(recom_j, orient='index', dtype=None, columns= columns_names)
five_recom

,Recommendation-1,Recommendation-2,Recommendation-3,Recommendation-4,Recommendation-5
61288,"Mushrooms - Black, Dried",Sponge Cake Mix - Chocolate,"Wine - Magnotta, Merlot Sr Vqa",Beef - Rib Eye Aaa,Chicken - Soup Base
77352,Guinea Fowl,Grenadine,Ecolab - Mikroklene 4/4 L,"Coconut - Shredded, Sweet","Beets - Candy Cane, Organic"
40094,Pasta - Orecchiette,Quiche Assorted,"Oregano - Dry, Rubbed",Tea - Decaf Lipton,"Liners - Banana, Paper"
23548,Spinach - Baby,Banana Turning,Tea - English Breakfast,Flavouring - Orange,Sauce - Demi Glace
78981,Lettuce - Frisee,Longos - Chicken Wings,Pop Shoppe Cream Soda,Crush - Cream Soda,Sprouts - Alfalfa
83106,Skirt - 29 Foot,"Cheese - Boursin, Garlic / Herbs",Garlic - Peeled,Soup - Campbells Tomato Ravioli,Beef - Rib Eye Aaa
11253,Smirnoff Green Apple Twist,Watercress,Tomatoes Tear Drop,Lettuce - Treviso,"Juice - Cranberry, 341 Ml"
35107,Vinegar - Sherry,Flavouring - Orange,Wonton Wrappers,Ice Cream Bar - Hageen Daz To,"Spoon - Soup, Plastic"
15088,Smirnoff Green Apple Twist,Blueberries,"Pepper - Black, Whole",Wonton Wrappers,Wine - Wyndham Estate Bin 777
26031,Muffin Batt - Choc Chk,Beef - Top Sirloin,Knife Plastic - White,Cheese - Mozzarella,Olive - Spread Tapenade
